This notebook produces the figures of the paper ... using the smaller data-files created from the full experiments outputs thanks to the notebook [data_files](data_files.ipynb).

In [ ]:
import os
import sys

import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
import cmocean.cm as cmo
import IPython
import matplotlib.pyplot as plt
import xarray as xr

In [ ]:
# Works in VS-Code only, hard-code it if necessary
nb_dir = "/".join(
    IPython.extract_module_locals()[1]["__vsc_ipynb_file__"].split("/")[:-1]
)
sys.path.append(os.path.dirname(nb_dir))

from domain import restrict_to_gulfstream

In [ ]:
CM_TO_IN = 1 / 2.54
MAX_WIDTH = 17.5 * CM_TO_IN
MAX_HEIGHT = 20.5 * CM_TO_IN
DPI = 300

SMALL_FONT_SIZE = 8
MEDIUM_FONT_SIZE = 10
LARGE_FONT_SIZE = 12

plt.rc("font", size=SMALL_FONT_SIZE)
plt.rc("axes", titlesize=SMALL_FONT_SIZE)
plt.rc("axes", labelsize=SMALL_FONT_SIZE)
plt.rc("xtick", labelsize=SMALL_FONT_SIZE)
plt.rc("ytick", labelsize=SMALL_FONT_SIZE)
plt.rc("legend", fontsize=SMALL_FONT_SIZE)

HW_SPACE = .33 * CM_TO_IN


def get_gridspec_kwargs(fig_width: float, fig_height: float) -> dict:
    return dict(wspace=HW_SPACE / fig_width, hspace=HW_SPACE / fig_height)


CLB_AXIS_HW = 1.25 * CM_TO_IN
CLB_BAR_HW = .33 * CM_TO_IN
CLB_HW_SPACE = .33 * CM_TO_IN


def get_clb_kwargs(axes_width: float, axes_height: float, clb_location: str, clb_shrink: float) -> dict:
    if clb_location in ("bottom", "top"):
        axes_hw_fraction = axes_height
        axes_hw_aspect = axes_width
    else:
        axes_hw_fraction = axes_width
        axes_hw_aspect = axes_height
    
    fraction = CLB_AXIS_HW / axes_hw_fraction
    aspect = clb_shrink * axes_hw_aspect / CLB_AXIS_HW
    pad = CLB_HW_SPACE / axes_hw_fraction

    return dict(location=clb_location, fraction=fraction, shrink=clb_shrink, aspect=aspect, pad=pad)


TITLE_H = .5 * CM_TO_IN

PLOT_GLOBAL_HW_RATIO = .42
PLOT_GS_HW_RATIO = .39

# Loading data-files

In [ ]:
# CAN / SHOULD BE UPDATED TO REPRODUCE

DATA_FILES_PATH = f"{nb_dir}/data-files"
FIGURES_PATH = f"{nb_dir}/figures"

# Cyclogeostrophic imbalance - Figure 1

In [ ]:
neurost_imbalance_uv_Geostrophy = xr.open_dataset(f"{DATA_FILES_PATH}/neurost_imbalance_uv_Geostrophy.nc")
neurost_imbalance_uv_Cyclogeostrophy = xr.open_dataset(f"{DATA_FILES_PATH}/neurost_imbalance_uv_Cyclogeostrophy.nc")
neurost_it_imbalance_uv_Cyclogeostrophy = xr.open_dataset(
    f"{DATA_FILES_PATH}/neurost_it_imbalance_uv_Cyclogeostrophy.nc"
)

In [ ]:
IMBALANCE_UV_MAX = 1.5


def plot_uv_imbalance(
    uv_imbalance: xr.DataArray, latitude: xr.DataArray, longitude: xr.DataArray, title: str, label: str, ax: GeoAxes
):
    im = ax.pcolormesh(
        longitude, latitude, uv_imbalance,
        cmap=cmo.amp, vmin=0, vmax=IMBALANCE_UV_MAX,
        transform=ccrs.PlateCarree()
    )
    ax.coastlines()

    ax.set_title(title, fontsize=MEDIUM_FONT_SIZE)
    ax.set_title(label, loc="left", fontsize=SMALL_FONT_SIZE)

    return im
    

In [ ]:
n_rows = 3
fig_width = MAX_WIDTH
fig_height = min(
    ((fig_width - (CLB_AXIS_HW + CLB_HW_SPACE)) * PLOT_GLOBAL_HW_RATIO + TITLE_H) * n_rows + HW_SPACE * (n_rows - 1),
    MAX_HEIGHT
)

fig, axd = plt.subplot_mosaic(
    """
    A
    B
    C
    """,
    subplot_kw=dict(projection=ccrs.PlateCarree()),
    gridspec_kw=get_gridspec_kwargs(fig_width, fig_height),
    figsize=(fig_width, fig_height), dpi=DPI, layout="compressed"
)

im = plot_uv_imbalance(
    neurost_imbalance_uv_Geostrophy.imbalance_uv_Geostrophy,
    neurost_imbalance_uv_Geostrophy.latitude, neurost_imbalance_uv_Geostrophy.longitude,
    "Before inversion", "(a)",
    axd["A"]
)

_ = plot_uv_imbalance(
    neurost_imbalance_uv_Cyclogeostrophy.imbalance_uv_Cyclogeostrophy,
    neurost_imbalance_uv_Cyclogeostrophy.latitude, neurost_imbalance_uv_Cyclogeostrophy.longitude,
    "Variational method", "(b)",
    axd["B"]
)

_ = plot_uv_imbalance( 
    neurost_it_imbalance_uv_Cyclogeostrophy.imbalance_uv_Cyclogeostrophy,
    neurost_it_imbalance_uv_Cyclogeostrophy.latitude, neurost_it_imbalance_uv_Cyclogeostrophy.longitude,
    "Iterative method", "(c)",
    axd["C"]
)

clb = fig.colorbar(
    im, ax=axd.values(),
    **get_clb_kwargs(fig_width, fig_height, clb_location="right", clb_shrink=.8), 
    extend="max"
)
clb.set_label(label="Cyclogeostrophic imbalance (m s$^{-1}$)", size=SMALL_FONT_SIZE)
tick_labels = [0.2, 0.6 , 1.0, 1.4]
clb.set_ticks(tick_labels)
clb.set_ticklabels(tick_labels, fontsize=SMALL_FONT_SIZE)

plt.savefig(f"{FIGURES_PATH}/fig1-neurost_imbalance_uv-V1.png", dpi=DPI, bbox_inches="tight")

plt.show()

In [ ]:
n_rows = 3
fig_width = MAX_WIDTH
fig_height = min(
    (fig_width * PLOT_GLOBAL_HW_RATIO + TITLE_H) * n_rows + HW_SPACE * (n_rows - 1) + CLB_AXIS_HW + CLB_HW_SPACE,
    MAX_HEIGHT
)

fig, axd = plt.subplot_mosaic(
    """
    A
    B
    C
    """,
    subplot_kw=dict(projection=ccrs.PlateCarree()),
    gridspec_kw=get_gridspec_kwargs(fig_width, fig_height),
    figsize=(fig_width, fig_height), dpi=DPI, layout="compressed"
)

im = plot_uv_imbalance(
    neurost_imbalance_uv_Geostrophy.imbalance_uv_Geostrophy,
    neurost_imbalance_uv_Geostrophy.latitude, neurost_imbalance_uv_Geostrophy.longitude,
    "Before inversion", "(a)",
    axd["A"]
)

_ = plot_uv_imbalance(
    neurost_imbalance_uv_Cyclogeostrophy.imbalance_uv_Cyclogeostrophy,
    neurost_imbalance_uv_Cyclogeostrophy.latitude, neurost_imbalance_uv_Cyclogeostrophy.longitude,
    "Variational method", "(b)",
    axd["B"]
)

_ = plot_uv_imbalance( 
    neurost_it_imbalance_uv_Cyclogeostrophy.imbalance_uv_Cyclogeostrophy,
    neurost_it_imbalance_uv_Cyclogeostrophy.latitude, neurost_it_imbalance_uv_Cyclogeostrophy.longitude,
    "Iterative method", "(c)",
    axd["C"]
)

clb = fig.colorbar(
    im, ax=axd.values(),
    **get_clb_kwargs(fig_width, fig_height, clb_location="bottom", clb_shrink=.8), 
    extend="max"
)
clb.set_label(label="Cyclogeostrophic imbalance (m s$^{-1}$)", size=SMALL_FONT_SIZE)
tick_labels = [0.2, 0.6 , 1.0, 1.4]
clb.set_ticks(tick_labels)
clb.set_ticklabels(tick_labels, fontsize=SMALL_FONT_SIZE)

plt.savefig(f"{FIGURES_PATH}/fig1-neurost_imbalance_uv-V2.png", dpi=DPI, bbox_inches="tight")

plt.show()

In [ ]:
n_rows = 2
n_cols = 2
fig_width = MAX_WIDTH
fig_height = min(
    (
        ((fig_width - HW_SPACE * (n_cols - 1)) / n_cols * PLOT_GLOBAL_HW_RATIO + TITLE_H) * n_rows + 
        HW_SPACE * (n_rows - 1) + CLB_AXIS_HW + CLB_HW_SPACE
    ),
    MAX_HEIGHT
)

fig, axd = plt.subplot_mosaic(
    """
    ..AAAA..
    BBBBCCCC
    """,
    subplot_kw=dict(projection=ccrs.PlateCarree()),
    gridspec_kw=get_gridspec_kwargs(fig_width, fig_height),
    figsize=(fig_width, fig_height), dpi=DPI, layout="compressed"
)

im = plot_uv_imbalance(
    neurost_imbalance_uv_Geostrophy.imbalance_uv_Geostrophy,
    neurost_imbalance_uv_Geostrophy.latitude, neurost_imbalance_uv_Geostrophy.longitude,
    "Before inversion", "(a)",
    axd["A"]
)

_ = plot_uv_imbalance(
    neurost_imbalance_uv_Cyclogeostrophy.imbalance_uv_Cyclogeostrophy,
    neurost_imbalance_uv_Cyclogeostrophy.latitude, neurost_imbalance_uv_Cyclogeostrophy.longitude,
    "Variational method", "(b)",
    axd["B"]
)

_ = plot_uv_imbalance( 
    neurost_it_imbalance_uv_Cyclogeostrophy.imbalance_uv_Cyclogeostrophy,
    neurost_it_imbalance_uv_Cyclogeostrophy.latitude, neurost_it_imbalance_uv_Cyclogeostrophy.longitude,
    "Iterative method", "(c)",
    axd["C"]
)

clb = fig.colorbar(
    im, ax=axd.values(), 
    **get_clb_kwargs(fig_width, fig_height, clb_location="bottom", clb_shrink=.8), 
    extend="max"
)
clb.set_label(label="Cyclogeostrophic imbalance (m s$^{-1}$)", size=SMALL_FONT_SIZE)
tick_labels = [0.2, 0.6 , 1.0, 1.4]
clb.set_ticks(tick_labels)
clb.set_ticklabels(tick_labels, fontsize=SMALL_FONT_SIZE)

plt.savefig(f"{FIGURES_PATH}/fig1-neurost_imbalance_uv-V3.png", dpi=DPI, bbox_inches="tight")

plt.show()

# EKE gain / loss - Figure 2

In [ ]:
neurost_eke_diff_rel_Cyclogeostrophy_Geostrophy = xr.open_dataset(
    f"{DATA_FILES_PATH}/neurost_eke_diff_rel_Cyclogeostrophy_Geostrophy.nc"
)

In [ ]:
EKE_DIFF_REL_MAX = 20

In [ ]:
fig_width = MAX_WIDTH
fig_height = min(fig_width * PLOT_GLOBAL_HW_RATIO + CLB_AXIS_HW + CLB_HW_SPACE, MAX_HEIGHT)

fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()), figsize=(fig_width, fig_height))

im = ax.pcolormesh(
    neurost_eke_diff_rel_Cyclogeostrophy_Geostrophy.longitude, neurost_eke_diff_rel_Cyclogeostrophy_Geostrophy.latitude, 
    neurost_eke_diff_rel_Cyclogeostrophy_Geostrophy.eke_diff_rel_Cyclogeostrophy_Geostrophy,
    cmap=cmo.balance_r, vmin=-EKE_DIFF_REL_MAX, vmax=EKE_DIFF_REL_MAX,
    transform=ccrs.PlateCarree()
)
ax.coastlines()

clb = fig.colorbar(
    im, ax=ax, 
    **get_clb_kwargs(fig_width, fig_height, clb_location="bottom", clb_shrink=.8), 
    extend="both"
)
clb.set_label(label="Gain (+) / loss (-) in Eddy Kinetic Energy (%)", size=SMALL_FONT_SIZE)
tick_labels = [-20, -10, 0, 10, 20]
clb.set_ticks(tick_labels)
clb.set_ticklabels(tick_labels, fontsize=SMALL_FONT_SIZE)

plt.savefig(f"{FIGURES_PATH}/fig2-neurost_eke_diff_rel_Cyclogeostrophy_Geostrophy.png", dpi=DPI, bbox_inches="tight")

plt.show()

# RMSE gain / loss - Figure 3

In [ ]:
neurost_spatial_err_sd_diff_Geostrophy_Cyclogeostrophy = xr.open_dataset(
    f"{DATA_FILES_PATH}/neurost_spatial_err_sd_diff_Geostrophy_Cyclogeostrophy.nc"
)

In [ ]:
RMSE_DIFF_REL_MAX = 15

In [ ]:
fig_width = MAX_WIDTH
fig_height = min(fig_width * PLOT_GLOBAL_HW_RATIO + CLB_AXIS_HW + CLB_HW_SPACE, MAX_HEIGHT)

fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()), figsize=(fig_width, fig_height))

im = ax.pcolormesh(
    neurost_spatial_err_sd_diff_Geostrophy_Cyclogeostrophy.longitude, 
    neurost_spatial_err_sd_diff_Geostrophy_Cyclogeostrophy.latitude, 
    neurost_spatial_err_sd_diff_Geostrophy_Cyclogeostrophy.err_sd_diff_Geostrophy_Cyclogeostrophy,
    cmap=cmo.balance_r, vmin=-RMSE_DIFF_REL_MAX, vmax=RMSE_DIFF_REL_MAX,
    transform=ccrs.PlateCarree()
)
ax.coastlines()

clb = fig.colorbar(
    im, ax=ax, 
    **get_clb_kwargs(fig_width, fig_height, clb_location="bottom", clb_shrink=.8), 
    extend="both"
)
clb.set_label(label="Gain (+) / loss (-) in RMSE (%)", size=SMALL_FONT_SIZE)
tick_labels = [-20, -10, 0, 10, 20]
clb.set_ticks(tick_labels)
clb.set_ticklabels(tick_labels, fontsize=SMALL_FONT_SIZE)

plt.savefig(
    f"{FIGURES_PATH}/fig3-neurost_err_mean_diff_rel_Geostrophy_Cyclogeostrophy.png", dpi=DPI, bbox_inches="tight"
)

plt.show()

# RMSE / EKE - Figure 4

In [ ]:
neurost_eke_binned_errors = xr.open_dataset(f"{DATA_FILES_PATH}/neurost_eke_binned_errors.nc")
duacs_eke_binned_errors = xr.open_dataset(f"{DATA_FILES_PATH}/duacs_eke_binned_errors.nc")

In [ ]:
fig, ax = plt.subplots()

ax.plot(
    neurost_eke_binned_errors.eke, neurost_eke_binned_errors.err_sd_Cyclogeostrophy, label="Neurost - Cyclogeostrophy"
)
ax.plot(neurost_eke_binned_errors.eke, neurost_eke_binned_errors.err_sd_Geostrophy, label="Neurost - Geostrophy")
ax.plot(duacs_eke_binned_errors.eke, duacs_eke_binned_errors.err_sd_Cyclogeostrophy, label="DUACS - Cyclogeostrophy")
ax.plot(duacs_eke_binned_errors.eke, duacs_eke_binned_errors.err_sd_Geostrophy, label="DUACS - Geostrophy")

ax.legend()

ax.set_xlabel("EKE quantile (m$^2$ s$^{-2}$)", size=SMALL_FONT_SIZE)
ax.set_ylabel("RMSE standard deviation (m s$^{-1}$)", size=SMALL_FONT_SIZE)

plt.savefig(
    f"{FIGURES_PATH}/fig4-eke_binned_rmse.png", dpi=DPI, bbox_inches="tight"
)

plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(duacs_eke_binned_errors.eke, duacs_eke_binned_errors.err_sd_Cyclogeostrophy, label="DUACS - Cyclogeostrophy")
ax.plot(duacs_eke_binned_errors.eke, duacs_eke_binned_errors.err_sd_Geostrophy, label="DUACS - Geostrophy")

ax.legend()

ax.set_xlabel("EKE quantile (m$^2$ s$^{-2}$)", size=SMALL_FONT_SIZE)
ax.set_ylabel("RMSE standard deviation (m s$^{-1}$)", size=SMALL_FONT_SIZE)

plt.show()

# Snapshot - Figure 5

In [ ]:
neurost_snapshot = xr.open_dataset(f"{DATA_FILES_PATH}/neurost_snapshot.nc")
duacs_snapshot = xr.open_dataset(f"{DATA_FILES_PATH}/duacs_snapshot.nc")

In [ ]:
neurost_snapshot_zoom = restrict_to_gulfstream(neurost_snapshot)
duacs_snapshot_zoom = restrict_to_gulfstream(duacs_snapshot)

In [ ]:
UV_MAX = 1.5
NRV_MAX = .5

def plot_nrv(
    nrv: xr.DataArray, latitude: xr.DataArray, longitude: xr.DataArray, label: str, ax: GeoAxes
):
    im = ax.pcolormesh(
        longitude, latitude, nrv,
        cmap=cmo.curl, vmin=-NRV_MAX, vmax=NRV_MAX,
        transform=ccrs.PlateCarree()
    )
    ax.coastlines()

    ax.set_title(label, loc="left", fontsize=MEDIUM_FONT_SIZE)

    return im

In [ ]:
n_rows = 3
n_cols = 2
fig_width = MAX_WIDTH
fig_height = min(
    (
        ((fig_width - (CLB_AXIS_HW + CLB_HW_SPACE)) * PLOT_GLOBAL_HW_RATIO + TITLE_H) +
        (
            (fig_width - HW_SPACE * (n_cols - 1) - (CLB_AXIS_HW + CLB_HW_SPACE)) / n_cols * PLOT_GS_HW_RATIO + TITLE_H
        ) * (n_rows - 1) + 
        HW_SPACE * (n_rows - 1)
    ),
    MAX_HEIGHT
)

fig, axd = plt.subplot_mosaic(
    """
    AA
    BC
    DE
    """,
    height_ratios=[2, 1, 1],
    subplot_kw=dict(projection=ccrs.PlateCarree()),
    gridspec_kw=get_gridspec_kwargs(fig_width, fig_height),
    figsize=(fig_width, fig_height), dpi=DPI, layout="compressed"
)

im_magn = axd["A"].pcolormesh(
    neurost_snapshot.longitude, neurost_snapshot.latitude, 
    neurost_snapshot.magn_Cyclogeostrophy,
    cmap=cmo.speed, vmin=0, vmax=UV_MAX,
    transform=ccrs.PlateCarree()
)
axd["A"].set_global()
axd["A"].coastlines()
axd["A"].set_title("(a)", loc="left", fontsize=MEDIUM_FONT_SIZE)

clb_magn = fig.colorbar(
    im_magn, ax=axd["A"],
    **get_clb_kwargs(fig_width, fig_height / 2, clb_location="right", clb_shrink=.8), 
    extend="both"
)
clb_magn.set_label(label="Surface current (m s$^{-1}$)", size=SMALL_FONT_SIZE)
tick_labels = [0.2, 0.6 , 1.0, 1.4]
clb_magn.set_ticks(tick_labels)
clb_magn.set_ticklabels(tick_labels, fontsize=SMALL_FONT_SIZE)

im_nrv = plot_nrv(
    neurost_snapshot_zoom.nrv_Cyclogeostrophy,
    neurost_snapshot_zoom.latitude, neurost_snapshot_zoom.longitude,
    "(b)",
    axd["B"]
)

_ = plot_nrv(
    neurost_snapshot_zoom.nrv_Geostrophy,
    neurost_snapshot_zoom.latitude, neurost_snapshot_zoom.longitude,
    "(c)",
    axd["C"]
)

_ = plot_nrv(
    duacs_snapshot_zoom.nrv_Cyclogeostrophy,
    duacs_snapshot_zoom.latitude, duacs_snapshot_zoom.longitude,
    "(d)",
    axd["D"]
)

_ = plot_nrv(
    duacs_snapshot_zoom.nrv_Geostrophy,
    duacs_snapshot_zoom.latitude, duacs_snapshot_zoom.longitude,
    "(e)",
    axd["E"]
)

clb_nrv = fig.colorbar(
    im_nrv, ax=list(axd.values())[1:],
    **get_clb_kwargs(fig_width, fig_height / 2, clb_location="right", clb_shrink=.8), 
    extend="both"
)
clb_nrv.set_label(label="Relative vorticity (f)", size=SMALL_FONT_SIZE)
tick_labels = [-.4, -.2, 0, .2, .4]
clb_nrv.set_ticks(tick_labels)
clb_nrv.set_ticklabels(tick_labels, fontsize=SMALL_FONT_SIZE)

plt.savefig(f"{FIGURES_PATH}/fig5-snapshot.png", dpi=DPI, bbox_inches="tight")

plt.show()